In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

## AWGN 함수

In [4]:
def awgn_(input, symbols, dB_snr):
    snr = 10**(dB_snr/10)
    signal_power = 1
    awgn_power = signal_power / snr
    awgn = torch.sqrt(awgn_power / 2)*(
        torch.randn_like(input, dtype=torch.float32)+1j*torch.randn_like(symbols, dtype=torch.float32)
    )
    return input + awgn

# 1. Deep Learning Decoder

In [5]:
# QPSK Decoder Model
class QPSKDemodulationModel(nn.Module):
    def __init__(self):
        super(QPSKDemodulationModel, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [ ]:
symbols, bits = generate_qpsk_data()
noisy_symbols = symbols + 0.1 * (np.random.randn(len(symbols)) + 1j * np.random.randn(len(symbols)))  # 노이즈 추가
X = np.column_stack((noisy_symbols.real, noisy_symbols.imag))
y = bits

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PyTorch 텐서로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# 모델 초기화
model = QPSKDemodulationModel()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
epochs = 10
batch_size = 128
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

# 복조 수행
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predicted_bits = (predictions > 0.5).int()

# 정확도 평가
accuracy = accuracy_score(y_test.numpy().flatten(), predicted_bits.numpy().flatten())
print(f"복조 정확도: {accuracy:.2f}")

In [ ]:
# BER 그래프
import matplotlib as plt

# BER 계산 함수
def calculate_ber(original_bits, predicted_bits):
    errors = (original_bits != predicted_bits).sum().item()
    ber = errors / original_bits.numel()
    return ber

# BER 그래프 작성 함수
def plot_ber(snr_range, ber_values):
    plt.figure(figsize=(8, 6))
    plt.plot(snr_range, ber_values, marker='o')
    plt.title("BER vs SNR")
    plt.xlabel("SNR (dB)")
    plt.ylabel("Bit Error Rate (BER)")
    plt.grid(True)
    plt.show()

# SNR에 따른 BER 계산 및 그래프 작성
def evaluate_and_plot_ber(model, X_test, y_test, snr_range):
    ber_values = []
    for snr_db in snr_range:
        # 노이즈 추가
        snr_linear = 10 ** (snr_db / 10)
        signal_power = torch.mean(torch.abs(X_test)**2)
        noise_power = signal_power / snr_linear
        noise = torch.sqrt(noise_power / 2) * torch.randn_like(X_test)
        noisy_test_signal = X_test + noise

        # 복조 수행
        model.eval()
        with torch.no_grad():
            predictions = model(noisy_test_signal)
            predicted_bits = (predictions > 0.5).int()

        # BER 계산
        ber = calculate_ber(y_test, predicted_bits)
        ber_values.append(ber)

    # BER 그래프 작성
    plot_ber(snr_range, ber_values)

# SNR 범위 설정
snr_range = range(0, 21, 2)  # 0 dB부터 20 dB까지 2 dB 간격

# BER 평가 및 그래프 작성
evaluate_and_plot_ber(model, X_test, y_test, snr_range)
